In [ ]:
import numpy as np
import pandas as pd
import os

import git
from git import RemoteProgress

from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [ ]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [ ]:
# uncomment the correct repo you want the information from



# # apache poi 
# remote_link = "https://github.com/apache/poi"
# local_link = "repo/poi"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "3a328aa220f6979f9805f658ae33244d153beaa7"
# affected_files = ["src/ooxml/java/org/apache/poi/POIXMLTypeLoader.java",
#                   "src/ooxml/java/org/apache/poi/util/DocumentHelper.java"]




# # cloudfoundry uua 
# remote_link = "https://github.com/cloudfoundry/uaa"
# local_link = "repo/uua"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "14c745aa293b8d3ce9cdd6bfbc6c0ef3f269b21"
# affected_files = ["server/src/main/java/org/cloudfoundry/identity/uaa/zone/GeneralIdentityZoneConfigurationValidator.java"]




# # apache struts 
# remote_link = "https://github.com/apache/struts"
# local_link = "repo/struts"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "74e26830d2849a84729b33497f729e0f033dc147"
# affected_files = ["core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java",
#                   "core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java"]



In [ ]:
def removeDuplicates(array):
    new_array = []
    for value in array:
        if value not in new_array:
            new_array.append(value)
    return new_array
        

In [ ]:
# prints the commit message, 3a

show_data = repo.git.show("-s", fixing_commit).splitlines()
for line in show_data:
    print(line)


In [ ]:
# print the number of files changed, 3b

# retuns the files that have been changed
files = repo.git.whatchanged("-1", "--format=oneline", fixing_commit).splitlines()

# minus one as one is the commit message
files.pop(0)

print(str(len(files)) + " files changed")

In [ ]:
# prints the number of unique directories, 3c

all_directories = []

for file in files:
    file_split = file.split()
    full_dir = file_split[5]
    directory = os.path.dirname(full_dir)
    all_directories.append(directory)
    
all_directories = removeDuplicates(all_directories)
print(str(len(all_directories)) + " directories changed")
    

In [ ]:
# prints the number of added/delete lines (inculding comments / blanks lines), 3d, 3e

lines_changed = repo.git.show("--pretty=tformat:", "--numstat", fixing_commit).splitlines()

total_lines_added = 0
total_lines_deleted = 0

for line in lines_changed:
    line_split = line.split()
    total_lines_added += int(line_split[0].strip())
    total_lines_deleted += int(line_split[1].strip())
    
print("Total deletions: " + str(total_lines_deleted))
print("Total additions: " + str(total_lines_added))

In [ ]:
def countNumberBlankComments(file, langauge):
    total_blank_additions = 0
    total_blank_deletions = 0
    
    if langauge == "Java":
        for line in file:
            if line[0] == "+":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_additions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_additions += 1
                    
            if line[0] == "-":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_deletions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_deletions += 1
    return total_blank_additions, total_blank_deletions

In [ ]:
# prints the number of added/deleted lines (Excluding comments / blanks lines) 3f, 3g

total_blank = [0, 0]

# for each changed file
for file in files:
    # get the directory
    file_split = file.split()
    full_dir = file_split[5]
    
    #get the diff and find the blanks and comments     
    affected_file_diff = repo.git.diff(fixing_commit + "^" + ".." + fixing_commit, "--", full_dir).splitlines()
    this_total_blank = countNumberBlankComments(affected_file_diff, "Java")
    
    # add blanks and comments to total     
    total_blank = np.add(total_blank, this_total_blank)

# get the total from before and the found blank lines and remove them from total
total_blank_additions = total_lines_added - total_blank[0]
total_blank_deletions = total_lines_deleted - total_blank[1]

print("Total deletions (without comments/blanklines): " + str(total_blank_deletions))
print("Total additions (without comments/blanklines): " + str(total_blank_additions))

In [ ]:
def convertToDate(input_date):
    split = input_date.split()
    year = split[4]
    month = strptime(split[1],'%b').tm_mon
    day = split[2]
    return date(year = int(year), month = month, day = int(day))

In [ ]:
# time difference between the fixing commit and the previous commit, 3h

for affected_file in affected_files:
    dates = repo.git.log("--format=%ad", fixing_commit, "--", affected_file).splitlines()

    date1_object = convertToDate(dates[0])
    if len(dates) < 2:
        print("no other changes have been made before")
    else:    
        date2_object = convertToDate(dates[1])
        total_days = abs((date1_object - date2_object).days)
        print("Total days since file", os.path.basename(affected_file), "changed:", total_days)

In [ ]:
# get the number of times this file has been changed, 3i

for affected_file in affected_files:
    times = repo.git.log("--format=%ad", fixing_commit, "--", affected_file).splitlines()
    print("This file", os.path.basename(affected_file), "has been changed:", len(times), "times")

In [ ]:
# what devolpers have modifed this file? 3j
all_files_authors = []
for affected_file in affected_files:
    all_authors = repo.git.log("--format=%an", fixing_commit, "--", affected_file).splitlines()
    authors = removeDuplicates(all_authors)
    
    all_files_authors.extend(authors)
    all_files_authors = removeDuplicates(all_files_authors)
    
    print(os.path.basename(affected_file), "\n\tAuthors:")
    for author in authors:
        print("\t\t", author)
    print("")

In [ ]:
def getAuthorCommits(author_name, df):
    row = df[df["Author"].str.contains(author_name)]
    commits = row["Commit"].iloc[0]
    return int(commits)

In [ ]:
# how many has each person commited

commit_authors = repo.git.shortlog("-sne", "--all").splitlines()

author_commits = []

for commit_author in commit_authors:
    commit, author = commit_author.split("\t")
    author_commits.append([int(commit), author])
    
df = pd.DataFrame(author_commits, columns=['Commit', 'Author'])

for author in all_files_authors:
    print(author, ":", getAuthorCommits(author, df), " Commits")

